# TlF B state eigenstates
In the Norrgard 2017 paper it is mentioned that the eigenstates of the B-state Hamiltonian are in fact superpositions of the uncoupled basis states |J, F1, F>. In this notebook I'm calculating the eigenstates by diagonalizing the Hamiltonian for the B state. The Hamiltonian is generated in the uncoupled basis ($\mathrm{|J, m_J, I_1, m_1, I_2, m_2>}$) so I'll also convert the relevant eigenstates into the coupled basis ($\mathrm{|J,I_1, I_2, F_1, F, m_F>})$ so that I can use them to calculate rotatinal branching ratios for decays from the B state to the groundstate.

In [22]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../molecular-state-classes-and-functions/')

import numpy as np
import sympy
import pickle
import matplotlib.pyplot as plt
%matplotlib notebook
from tqdm import tqdm_notebook

from classes import *
from functions import *


with open("hamiltonians_symbolic.py", 'rb') as f:
    hamiltonians = pickle.load(f)

Jmin = sympy.Integer(1)
Jmax = sympy.Integer(11) # max J value in Hamiltonian
I_Tl = 1/2             # I1 in Ramsey's notation/
I_F  = 1/2             # I2 in Ramsey's notation

def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(sympy.Number(x))
        x += dx
    return range_list

# QN = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2)
#       for J  in ni_range(Jmin, Jmax+1)
#       for mJ in ni_range(-J,J+1)
#       for m1 in ni_range(-I_Tl,I_Tl+1)
#       for m2 in ni_range(-I_F,I_F+1)
#      ]

QN = []

for J in np.arange(Jmin, Jmax+1):
    for m1 in np.arange(-I_Tl,I_Tl+1):
        for m2 in np.arange(-I_F,I_F+1):
            mJ = -m1-m2
            
            if np.abs(mJ) <= J:
                QN.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
%%time
variables = [
    sympy.symbols('Brot'),
    *sympy.symbols('h1_Tl h1_F'),
    *sympy.symbols('c1 c2 c3 c4'),
    sympy.symbols('D_TlF'),
    *sympy.symbols('mu_J mu_Tl mu_F')
]

lambdified_hamiltonians = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians.items()
}

Wall time: 145 ms


Insert numerical constants into Hamiltonian (all in units of Hz):

In [24]:
%%time
Brot = 6689920000
h1_Tl = 28793e6
h1_F = 871e6
#h1_F = 0
c1 = 0
c2 = 0
c3 = 0
c4 = 0
D_TlF = 4.2282 * 0.393430307 *5.291772e-9/4.135667e-15 # [Hz/(V/cm)]
mu_J = 35
mu_Tl = 1240.5
mu_F = 2003.63

H = {
    H_name : H_fn(
        Brot,
        h1_Tl, h1_F,
        c1, c2, c3, c4,
        D_TlF,
        mu_J, mu_Tl, mu_F
    )
    for H_name, H_fn in lambdified_hamiltonians.items()
}

Wall time: 997 µs


In [25]:
#Find the eigenstates in zero field
def spectrum(Ex_arr,Ey_arr,Ez_arr,Bx_arr,By_arr,Bz_arr):
    energies_arr = []
    energies_rot_arr = []
    states_arr = []
    for Ex,Ey,Ez,Bx,By,Bz in zip(Ex_arr,Ey_arr,Ez_arr,Bx_arr,By_arr,Bz_arr):
        HamE = H["Hff"]
        D, V = np.linalg.eigh(HamE)
        idx = D.argsort() 
        D = D[idx]
        V = V[:,idx]
        
        # Subtract away rotational energy for easier viewing of substructure
        hfs_mat = []
        for i,state in enumerate(QN):
            J = state.J
            hfs_mat.append(D[i] - J*(J +1)*Brot)
        hfs_kHz = np.array(hfs_mat)/1000

        energies_arr.append(hfs_kHz)
        energies_rot_arr.append(D)
        states_arr.append(V)
    return np.array(energies_arr), np.array(energies_rot_arr), np.array(states_arr)

Ez = np.zeros(1)
Bz = np.zeros(Ez.shape)
Ex = np.zeros(Ez.shape)
Ey = np.zeros(Ez.shape)
Bx = np.zeros(Ez.shape)
By = np.zeros(Ez.shape)

energies,eigenstates_mat, = spectrum(Ex,Ey,Ez,Bx,By,Bz)[1:]



In [26]:
# Convert the matrix of eigenvectors into a list of eigenstates
eigenstates_uncoupled = matrix_to_states(eigenstates_mat[0,:,:], QN, E = energies[0,:])

In [28]:
#Convert the states into the coupled basis
eigenstates_coupled = []
for eigenstate_uncoupled in tqdm_notebook(eigenstates_uncoupled):
    eigenstate_coupled = eigenstate_uncoupled.remove_small_components().transform_to_coupled()
    eigenstate_coupled.energy = eigenstate_uncoupled.energy
    eigenstates_coupled.append(eigenstate_coupled)
    
    

In [30]:
eigenstate_dict = {}


for eigenstate in eigenstates_coupled:
    print("")
    string = eigenstate.find_largest_component().print_quantum_numbers()
    print("=")
    eigenstate.remove_small_components().print_state()
    
    print("E = {:.3E}".format(eigenstate.energy))
    
    eigenstate_dict[string] = eigenstate


|J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2>
=
+1.0000 x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2>
E = -1.452E+09

|J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
=
-0.9996 x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
-0.0203 x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
-0.0180 x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
E = -8.821E+08

|J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
=
+0.0267 x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
-0.8519 x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
-0.5231 x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2>
E = 1.264E+10

|J = 1, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2>
=
-0.8483 x |J = 1, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2>
-0.5293 x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2>
-0.0138 x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2>
-0.0064 x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2>
E = 1.296E+10

|J = 2, F1 = 5/2, F = 2,

In [31]:
with open("B_eigenstates.py", 'wb') as f:
    pickle.dump(eigenstate_dict,f)